In [1]:
#imports
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
#fetch url
url = "https://www.mediaite.com/daily-ratings/cable-news-ratings-thursday-june-8-msnbc-sweeps-fox-and-cnn-in-prime-time-with-trump-indictment-coverage/"
base = requests.get(url)
base

<Response [200]>

In [3]:
#convert to html text & test
base_html = base.text 
print(base_html)

<!DOCTYPE html><html lang="en"><head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=5, minimum-scale=1.0">
<meta name="date" content="2023-06-09T16:33:25+00:00">
<!-- Open Graph -->
<!-- End Open Graph -->
<link rel="shortcut icon" href="https://www.mediaite.com/favicon.ico" /><link rel="apple-touch-icon" href="https://am12.mediaite.com/med/cnt/themes/m2019/images/icon-192x192.png"><link rel="icon" sizes="192x192" href="https://am12.mediaite.com/med/cnt/themes/m2019/images/icon-192x192.png">
		<!-- All in One SEO 4.4.0.1 - aioseo.com -->
		<title>Cable News Ratings Thursday June 8: MSNBC, CNN &amp; Fox News</title>
		<meta name="description" content="Chris Hayes, Alex Wagner, and Lawrence O&#039;Donnell all won their respective hours between 8 and 11 p.m. in both total viewers and in the key 25-54 age demographic." />
		<meta name="robots" content="max-snippet:-1, max-image-preview:large, max-video-preview:-1" />
		<meta name

In [4]:
#make soup object
soup = BeautifulSoup(base_html)
type(soup)

bs4.BeautifulSoup

In [5]:
#selecting just the first of two tables on the page
table = soup.select("table")[0]

In [6]:
#creating header list
first_row = table.select("tbody tr")[0]
header_list = [ cell.text.rstrip("\n") for cell in first_row.select("td") ]
header_list

['ET', 'FNC', 'CNN', 'MSNBC', 'NEWSMAX', 'NEWSNATION']

In [7]:
#selecting non header rows of data from table 
rows_ratings = table.select("tr")[1:19]
len(rows_ratings)
print(rows_ratings)

[<tr>
<td data-sheets-value='{"1":2,"2":"6a"}'>6a</td>
<td data-sheets-value='{"1":2,"2":"FOX AND FRIENDS:\n844"}'>FOX AND FRIENDS:<br/>
844</td>
<td data-sheets-value='{"1":2,"2":"CNN THIS MORNING:\n262"}'>CNN THIS MORNING:<br/>
262</td>
<td data-sheets-value='{"1":2,"2":"MORNING JOE:\n1059"}'>MORNING JOE:<br/>
1059</td>
<td data-sheets-value='{"1":2,"2":"DICK MORRIS DEMOCRACY:\n53"}'>DICK MORRIS DEMOCRACY:<br/>
53</td>
<td data-sheets-value='{"1":2,"2":"EARLY MORNING:\n25"}'>EARLY MORNING:<br/>
25</td>
</tr>, <tr>
<td data-sheets-value='{"1":2,"2":"7a"}'>7a</td>
<td data-sheets-value='{"1":2,"2":"FOX AND FRIENDS:\n1106"}'>FOX AND FRIENDS:<br/>
1106</td>
<td data-sheets-value='{"1":2,"2":"CNN THIS MORNING:\n322"}'>CNN THIS MORNING:<br/>
322</td>
<td data-sheets-value='{"1":2,"2":"--"}'>—</td>
<td data-sheets-value='{"1":2,"2":"WAKE UP AMERICA:\n211"}'>WAKE UP AMERICA:<br/>
211</td>
<td data-sheets-value='{"1":2,"2":"MORNING IN AMERICA:\n36"}'>MORNING IN AMERICA:<br/>
36</td>
</tr>, <t

In [8]:
#creating a list of cells from the table
rating_entries = []
for row in rows_ratings:
    row_cells = []
    for cell in row.select("td"):
        row_cells.append(cell.text.rstrip("\n"))
    rating_entries.append(row_cells)
rating_entries[:3]

[['6a',
  'FOX AND FRIENDS:\n844',
  'CNN THIS MORNING:\n262',
  'MORNING JOE:\n1059',
  'DICK MORRIS DEMOCRACY:\n53',
  'EARLY MORNING:\n25'],
 ['7a',
  'FOX AND FRIENDS:\n1106',
  'CNN THIS MORNING:\n322',
  '—',
  'WAKE UP AMERICA:\n211',
  'MORNING IN AMERICA:\n36'],
 ['8a', 'FOX AND FRIENDS:\n1265', 'CNN THIS MORNING:\n418', '—', '—', '—']]

In [9]:
cable8_df = pd.DataFrame(rating_entries, columns=header_list)
cable8_df.head()

,ET,FNC,CNN,MSNBC,NEWSMAX,NEWSNATION
0,6a,FOX AND FRIENDS:\n844,CNN THIS MORNING:\n262,MORNING JOE:\n1059,DICK MORRIS DEMOCRACY:\n53,EARLY MORNING:\n25
1,7a,FOX AND FRIENDS:\n1106,CNN THIS MORNING:\n322,—,WAKE UP AMERICA:\n211,MORNING IN AMERICA:\n36
2,8a,FOX AND FRIENDS:\n1265,CNN THIS MORNING:\n418,—,—,—
3,9a,AMERICAS NEWSROOM:\n1320,CNN NEWS CENTRAL:\n448,MORNING JOE:\n937,NATIONAL REPORT:\n203,—
4,10a,AMERICAS NEWSROOM:\n1322,CNN NEWS CENTRAL:\n530,ANA CABRERA REPORTS:\n710,—,NWSN LIVE W/ MARNI HUGHES:\n29


In [10]:
#dropping everything but fox to create fox column only df
fox8 = cable8_df.drop(columns=['CNN', 'MSNBC','NEWSMAX',"NEWSNATION","ET"])
fox8

,FNC
0,FOX AND FRIENDS:\n844
1,FOX AND FRIENDS:\n1106
2,FOX AND FRIENDS:\n1265
3,AMERICAS NEWSROOM:\n1320
4,AMERICAS NEWSROOM:\n1322
5,"FAULKNER FOCUS, THE:\n1295"
6,OUTNUMBERED:\n1432
7,AMERICA REPORTS:\n1247
8,AMERICA REPORTS:\n1119
9,"STORY, THE:\n1170"


In [11]:
#splitting the string and numbers in the FNC column, while dropping the :\n . And then dropping the original FNC Column
fox8[['Show', '6/8']] = fox8['FNC'].str.split(':\n', expand=True)
fox8 = fox8.drop(columns=['FNC'])
fox8

,Show,6/8
0,FOX AND FRIENDS,844
1,FOX AND FRIENDS,1106
2,FOX AND FRIENDS,1265
3,AMERICAS NEWSROOM,1320
4,AMERICAS NEWSROOM,1322
5,"FAULKNER FOCUS, THE",1295
6,OUTNUMBERED,1432
7,AMERICA REPORTS,1247
8,AMERICA REPORTS,1119
9,"STORY, THE",1170


In [12]:
fox8.to_csv("fox8.csv")